# 6. Hypothesis CFP ESG Portfolios
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
We estimate equation 4 for Low, High and for Low-High

In [2]:
import pandas as pd
import ast
import glob
import numpy as np

import psycopg2 
import statsmodels.formula.api as smf
import pandas_datareader.data as web
import statsmodels.api as sm

from sqlalchemy import create_engine 
from datetime import timedelta, datetime

In [3]:
credentials = ast.literal_eval(open(r"C:\Users\pvilas\OneDrive - unizar.es\Python Investigación\credentials.txt", "r").read())
engine = create_engine(credentials['sqlalchemy'][0]+ 'ESGRefinitiv')

In [4]:
df_market = pd.read_sql('a12_daily_mv_clean', engine)
df_market['yearly_date'] = df_market['daily_date'].dt.isocalendar().year

df_market = df_market.set_index(['internal_id', 'daily_date']).sort_index()
# drop observations when the price_index was the same in the last 126 observations and the price_index is less than one
df_market['drop'] = df_market.groupby(level='internal_id')['price_index'].rolling(126).apply(
    lambda x: 1 if len(x[x == x[0]])==126 else 0).reset_index(0,drop=True) 
df_market = df_market[(df_market['price_index']>1) & (df_market['drop']!=1)]

df_market['Return'] = df_market.groupby(level='internal_id')['return_index_usd'].pct_change()
df_market['Return'] = df_market['Return']*100
df_market['Return'] = df_market[['Return']].apply(lambda x: x.clip(x.quantile(0.01), x.quantile(0.99)))

In [5]:
# Create High and Low ESG portfolio
query_sql = """
SELECT internal_id, year_score,  esg_score_average, environment_pillar_average, social_pillar_average, 
governance_pillar_average,  overall_inclusive, environmental_inclusive, social_inclusive, governance_inclusive, 
ds_country
FROM 
b31_accounting_isp_ssp
"""
df_b31 = pd.read_sql(query_sql, engine)
df_b31['year_score'] = df_b31['year_score'].astype(int)

df_data = df_market.reset_index().merge(df_b31, 
                        right_on=['year_score', 'internal_id'], left_on = ['yearly_date', 'internal_id'])

In [6]:
df = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', '01-01-2000', '30-11-2021')[0].replace({
                            -99.99: np.nan, -999:np.nan})

df.insert(0, 'Alpha', 1)
df = df[['Alpha', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]


scores = {
    'conventional':['esg_score_average','environment_pillar_average','social_pillar_average','governance_pillar_average'],
'inclsuive':['overall_inclusive', 'environmental_inclusive', 'social_inclusive', 'governance_inclusive'] }
cuts = [5,10,20]
df_results_conv = pd.DataFrame()
df_results_incl = pd.DataFrame()
results = {'conventional': df_results_conv, 'inclsuive':df_results_incl}


for kind in scores:
    for score in scores[kind]:
        df_results_score = pd.DataFrame()
        for cut in cuts:
            df_return = df_data.copy()
            if score=='environment_pillar_average': 
                df_return = df_return[df_return[score]!=0]
                df_return['score_cut'] = df_return.groupby('year_score')[score].apply(lambda x: pd.qcut(x, cut, labels=False))
            else:
                df_return['score_cut'] = df_return.groupby('year_score')[score].apply(lambda x: pd.qcut(x, cut, labels=False))
    
            df_return = df_return.pivot_table(columns='score_cut', values='Return', index='daily_date', aggfunc='mean')
            df_return = df_return[[0, cut-1]].rename(columns={0:'Low', cut-1:'High'})
            df_return['High-Low'] = df_return['High']-df_return['Low']

            for portfolio in ['Low', 'High', 'High-Low']: 
                df_return[portfolio] = df_return[portfolio] - df['RF']
                df_reg = df.drop(columns='RF').merge(df_return[portfolio], right_index=True, left_index=True)
                res = sm.OLS(df_reg.iloc[:,-1:],df_reg.iloc[:,:-1]).fit()
                res = results_research(res,[] ,[[kind],[cut],[portfolio]])
                res.columns.set_names(['kind', '#Partitions', 'Portfolio'], inplace=True)
                df_results_score = pd.concat([df_results_score, res], axis=1)
                
        df_results_score['score'] = score
        df_results_score = df_results_score.set_index('score', append=True).reorder_levels([1,0], axis=0)
        results[kind] = pd.concat([results[kind], df_results_score])

results[kind].loc[(slice(None),('Alpha','R2')),:]

kind                                 inclsuive                   \
#Partitions                                  5                    
Portfolio                                  Low             High   
score                                                             
overall_inclusive       Alpha   0.016\n(0.009)  -0.004\n(0.010)   
environmental_inclusive Alpha  0.019*\n(0.009)  -0.004\n(0.010)   
social_inclusive        Alpha   0.014\n(0.010)  -0.003\n(0.010)   
governance_inclusive    Alpha   0.012\n(0.009)   0.001\n(0.010)   
overall_inclusive       R2              62.33%           60.72%   
environmental_inclusive R2              65.26%           62.06%   
social_inclusive        R2              61.48%           61.77%   
governance_inclusive    R2              64.13%           64.20%   

kind                                                               \
#Partitions                                                    10   
Portfolio                               High-Low              Low   
score                                                               
overall_inclusive       Alpha  -0.022**\n(0.003)  0.023*\n(0.010)   
environmental_inclusive Alpha  -0.025**\n(0.003)  0.023*\n(0.010)   
social_inclusive        Alpha  -0.019**\n(0.002)   0.016\n(0.010)   
governance_inclusive    Alpha  -0.014**\n(0.002)   0.018\n(0.009)   
overall_inclusive       R2                19.62%           61.01%   
environmental_inclusive R2                16.02%           61.13%   
social_inclusive        R2                13.83%           60.42%   
governance_inclusive    R2                29.42%           64.31%   

kind                                                               \
#Partitions                                                         
Portfolio                                 High           High-Low   
score                                                               
overall_inclusive       Alpha  -0.006\n(0.010)  -0.031**\n(0.004)   
environmental_inclusive Alpha  -0.006\n(0.010)  -0.032**\n(0.004)   
social_inclusive        Alpha  -0.006\n(0.010)  -0.024**\n(0.004)   
governance_inclusive    Alpha   0.000\n(0.010)  -0.020**\n(0.003)   
overall_inclusive       R2              58.82%             16.75%   
environmental_inclusive R2              64.91%             23.35%   
social_inclusive        R2              59.43%              7.92%   
governance_inclusive    R2              63.78%             29.05%   

kind                                                             \
#Partitions                                 20                    
Portfolio                                  Low             High   
score                                                             
overall_inclusive       Alpha  0.023*\n(0.010)  -0.007\n(0.011)   
environmental_inclusive Alpha  0.023*\n(0.010)  -0.009\n(0.010)   
social_inclusive        Alpha   0.018\n(0.011)  -0.008\n(0.011)   
governance_inclusive    Alpha  0.020*\n(0.009)  -0.007\n(0.010)   
overall_inclusive       R2              58.02%           57.17%   
environmental_inclusive R2              54.29%           66.00%   
social_inclusive        R2              57.06%           54.16%   
governance_inclusive    R2              67.58%           63.38%   

kind                                              
#Partitions                                       
Portfolio                               High-Low  
score                                             
overall_inclusive       Alpha  -0.032**\n(0.005)  
environmental_inclusive Alpha  -0.035**\n(0.005)  
social_inclusive        Alpha  -0.028**\n(0.005)  
governance_inclusive    Alpha  -0.029**\n(0.004)  
overall_inclusive       R2                12.28%  
environmental_inclusive R2                27.24%  
social_inclusive        R2                 6.86%  
governance_inclusive    R2                22.76%

In [7]:
results['conventional'].loc[(slice(None),('Alpha','R2')),:]

kind                                conventional                   \
#Partitions                                    5                    
Portfolio                                    Low             High   
score                                                               
esg_score_average          Alpha  0.008\n(0.010)  -0.001\n(0.011)   
environment_pillar_average Alpha  0.006\n(0.009)  -0.002\n(0.012)   
social_pillar_average      Alpha  0.008\n(0.011)  -0.001\n(0.011)   
governance_pillar_average  Alpha  0.010\n(0.009)   0.002\n(0.010)   
esg_score_average          R2             56.77%           58.52%   
environment_pillar_average R2             64.99%           53.72%   
social_pillar_average      R2             43.17%           61.57%   
governance_pillar_average  R2             64.37%           62.43%   

kind                                                                 \
#Partitions                                                      10   
Portfolio                                  High-Low             Low   
score                                                                 
esg_score_average          Alpha    -0.011\n(0.007)  0.007\n(0.011)   
environment_pillar_average Alpha    -0.009\n(0.007)  0.007\n(0.009)   
social_pillar_average      Alpha    -0.011\n(0.008)  0.011\n(0.013)   
governance_pillar_average  Alpha  -0.010**\n(0.003)  0.014\n(0.009)   
esg_score_average          R2                15.79%          46.84%   
environment_pillar_average R2                 8.83%          64.35%   
social_pillar_average      R2                23.34%          28.53%   
governance_pillar_average  R2                29.40%          63.25%   

kind                                                                  \
#Partitions                                                            
Portfolio                                    High           High-Low   
score                                                                  
esg_score_average          Alpha  -0.004\n(0.012)    -0.014\n(0.008)   
environment_pillar_average Alpha  -0.003\n(0.013)    -0.012\n(0.008)   
social_pillar_average      Alpha  -0.004\n(0.012)    -0.017\n(0.010)   
governance_pillar_average  Alpha  -0.002\n(0.010)  -0.018**\n(0.004)   
esg_score_average          R2              58.41%             19.86%   
environment_pillar_average R2              52.81%              8.67%   
social_pillar_average      R2              60.51%             29.41%   
governance_pillar_average  R2              60.52%             25.16%   

kind                                                               \
#Partitions                                   20                    
Portfolio                                    Low             High   
score                                                               
esg_score_average          Alpha  0.015\n(0.012)  -0.006\n(0.012)   
environment_pillar_average Alpha  0.009\n(0.010)  -0.003\n(0.014)   
social_pillar_average      Alpha  0.014\n(0.014)  -0.003\n(0.012)   
governance_pillar_average  Alpha  0.015\n(0.009)  -0.004\n(0.011)   
esg_score_average          R2             37.48%           57.41%   
environment_pillar_average R2             61.75%           52.07%   
social_pillar_average      R2             19.05%           59.49%   
governance_pillar_average  R2             62.08%           57.24%   

kind                                                 
#Partitions                                          
Portfolio                                  High-Low  
score                                                
esg_score_average          Alpha   -0.023*\n(0.009)  
environment_pillar_average Alpha    -0.014\n(0.010)  
social_pillar_average      Alpha    -0.019\n(0.012)  
governance_pillar_average  Alpha  -0.021**\n(0.005)  
esg_score_average          R2                25.70%  
environment_pillar_average R2                 9.65%  
social_pillar_average      R2                32.33%  
governance_pillar_average  R2 

In [8]:
results['conventional'].to_excel(r'C:\Users\pvilas\OneDrive - unizar.es\Tesis\2022 ISP\3. Tables & Figures\3. Definitive\Table 6 PortConventional.xlsx')
results['inclsuive'].to_excel(r'C:\Users\pvilas\OneDrive - unizar.es\Tesis\2022 ISP\3. Tables & Figures\3. Definitive\Table 7 PortInclusive.xlsx')

In [9]:
def results_research (res, effects, model_name):
        # coeficients results std_errors res.tstats
        df_results = pd.concat([res.params,res.bse, res.pvalues ], 
                               keys = ["Parameter", "Std.Err.", 'P-value'],  axis= 1)

        for effect in effects:
            effect =  effect.replace('C(','').replace(')','')
            df_results = df_results.filter(axis=0, regex ='^((?!'+ effect + ').)*$')
        
        # Results research format
        df_results =  ( df_results["Parameter"].round(3).map('{:.3f}'.format).astype(str) + 
                         df_results['P-value'].apply(significance) + 
                         df_results["Std.Err."].round(3).map('{:.3f}'.format).astype(str).apply(lambda x:  "("+ x + ")" ) 
                        )         

        for effect in effects:
            df_results[effect] = 'Yes'
            
        df_results['R2'] = '{:.2%}'.format(res.rsquared)   
        df_results = df_results.to_frame()  
        df_results.columns= model_name
        
        return df_results
    
def significance(p_val):
    significance = '{:.2f}'.format(p_val)
#     significance = significance if  significance[0]=='1' else significance[1:]
    if p_val <=0.05 and p_val > 0.01 :
        significance = '*'
    elif p_val <=0.01 and p_val > 0.001:
        significance = '**'
    elif p_val <=0.001 :
        significance = '**'
    else:
        significance= ''
    significance =  significance + "\n"
    return significance